In [1]:
#Mounting the drive content that contains the necessary files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#path of the file that contains the maxims to be augmented
TRAINSETPATHX='/content/drive/MyDrive/Maximsamples.txt'

f=open(TRAINSETPATHX,'r',encoding='utf8')
maximbits=f.readlines()
f.close()

#removing the endline characters
maximbits=[maxim[:-1] for maxim in maximbits]

print("The number of maxims: {}".format(len(maximbits)))

The number of maxims: 10


In [3]:
#importing the necessary modules
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
from nltk.corpus import stopwords,wordnet
import numpy as np
import re
import random

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [4]:
#obtaining the list of stop words in english
stop_words = set(stopwords.words('english'))

In [5]:
#function that finds the similar words to the input using the WordNet and POS tags on 2 different levels
def synonyms(wordgiven, ndesired):
    #collecting the synsets from the WordNet interface
    wn=wordnet.synsets(wordgiven)
    words=[]
    taglist=[]
    #obtaning the POS tag of the word in process
    wordtag=nltk.pos_tag([wordgiven])[0][1][0]
    check=1
    #if the result returned from the wordnet interface is not empty
    if len(wn)!=0:
        for word in wn:
            #performing a POS compatibility check on the synset level that result in a prioritization tag
            for entry in word.lemmas():
                if wordtag=='N' and str(entry).split('.')[1]=='n':
                    priortag=1
                elif (wordtag in ['J']) and (str(entry).split('.')[1] in ['a','s']):
                    priortag=1
                elif (wordtag in ['R']) and (str(entry).split('.')[1] in ['r']):
                    priortag=1
                else:
                    priortag=0
                if (wordgiven.lower()!=entry.name().lower()) and (entry.name().lower() not in words):
                    words.append(entry.name().lower()) #list that contains the results
                    taglist.append(priortag) #list that contains the prioritization tags of the first check
    #reordering of the results in the list based on the first prioritization tags obtained
    #and saving the rioritization tags of the second check
    wordsfinal=[None]*len(words) 
    if len(words)!=0:
        indexstart=taglist.count(1) 
        count1=0 
        indicate=[None]*len(words) #list that contains the prioritization tags of the second check
        for i in range(0,len(words)):
            if taglist[i]==1:
                wordsfinal[count1]=words[i]
                #determining the second indicators
                if wordtag==nltk.pos_tag([words[i]])[0][1][0]:
                    indicate[count1]=1
                else:
                    indicate[count1]=0
                count1+=1
            else:
                #placing the unprioritized words at the lower indices
                wordsfinal[indexstart]=words[i]
                if wordtag==nltk.pos_tag([words[i]])[0][1][0]:
                    indicate[indexstart]=1
                else:
                    indicate[indexstart]=0
                indexstart+=1
        #reordering of the results in the list based on the second prioritization tags obtained
        wordsfinal2=[None]*len(wordsfinal)
        indexstart1_2=indicate[0:taglist.count(1)].count(1) 
        count2=0 
        #reordering of the already prioritized words (at the first check) based on the second prioritization tags obtained
        for i in range(0,taglist.count(1)):
            if indicate[i]==1:
                wordsfinal2[count2]=wordsfinal[i]
                count2+=1
            else:
                wordsfinal2[indexstart1_2]=wordsfinal[i]
                indexstart1_2+=1
        #reordering of the unprioritized words (at the first check) based on the second prioritization tags obtained
        indexstart1_2=indicate[taglist.count(1):].count(1) 
        count2=taglist.count(1)
        for i in range(taglist.count(1),len(wordsfinal)):
            if indicate[i]==1:
                wordsfinal2[count2]=wordsfinal[i]
                count2+=1
            else:
                wordsfinal2[indexstart1_2]=wordsfinal[i]
                indexstart1_2+=1  
        check=1
    else:
        #returning an empty list if the number of results is equal to zero
        wordsfinal2=wordsfinal
        check=0
    return wordsfinal2[0:ndesired],check   

In [6]:
#function that cleans the lines from undesired characters as tags,punctuation signs etc.
def cleanline(line):
    tagpattern=r'(<)(.+?)(>)'
    numberpattern=r'([0-9]+)'
    cl=re.sub(tagpattern,'',line.lower()).replace('\n','').replace('“','').replace('”','').replace('…','').replace('—',' ').replace('‘','').replace('’','').replace('\\' , ' ').replace('/',' ')
    cl=re.sub(numberpattern,' ',cl)
    for char in cl:
            if (char in string.punctuation):
                cl=cl.replace(char,' ')
    return cl

In [7]:
#breaking th maxim lines into sentences
sentences=[]
nsentence=[] #storing the number of sentences each maxim contains
for line in maximbits:
    count=0
    #splitting sentences using ". " characters
    #listsent=line.replace('\\\n','').split('. ')
    listsent=line.split('. ')
    for i in range(0,len(listsent)):
        sentence=listsent[i]
        #sentences that are too short are eliminated
        if len(sentence)<5:
            continue
        elif (i==len(listsent)-1):
            #the last one is added as it is since we used the character sequence ". " for the seperation
            #and the last sentence contains only "."
            sentences.append(sentence)
            count=count+1
            continue
        else:
                        #the "." characters are restored
            if(sentence[-1]!='.'):
                sentence=sentence+'.'
            sentences.append(sentence)
            count=count+1
    nsentence.append(count)

In [8]:
print(len(sentences))
print(len(nsentence))

14
10


In [9]:
perfix=[0.6] #percentage desired
noisesent6=[]
for sent in sentences:
    print(sentences.index(sent))
    #tokinizing and capitalizing the sentences
    tokens = nltk.word_tokenize(cleanline(sent).capitalize())
    #lower-casing the tokens
    tokens=[token.lower() for token in tokens]
    #eliminating the stop words
    nonstoptoken = [word for word in tokens if not word in stop_words]
    tags = nltk.pos_tag(nonstoptoken)
    #collecting the tokens with POS tag in categories nouns, adjectives and adverbs
    tochange=[tag for tag in tags if tag[1][0] in ['J','N','R']]
    #print(tochange)
    #print("Len to change: {}".format(len(tochange)))
    nchange=[round(per*len(tochange)) for per in perfix] #number of required replacements due to the percentage given
    #print("Nchange: {}".format(nchange[0]))
    linemodtotal=sent.lower()
    indicator=sent[0].isupper()
    changed=0
    index=-1
    #the replacement order of the tokens could also be randomized by making the following line uncommented
    #random.shuffle(tochange)
    while(changed<nchange[-1]):
        index+=1
        if(index==len(tochange)):
            if changed<nchange[0]:
              if (indicator):
                  toadd=linemodtotal.replace('#','').capitalize()
              else:
                  toadd=linemodtotal.replace('#','')
              noisesent6.append(toadd)
              print(toadd)
            break
        word=tochange[index][0]
        #print("Word to change: {}".format(word))
        #Obtaining the candidates for the replacement
        [wn,check2]=synonyms(tochange[index][0],5)
        #if the list returned is not empty
        if (check2):
            #first candidate is chosen
            toreplace=wn[0]
            #if the candidate chosen contains the character "_" it is replaced with a space character
            if '_' in toreplace:
                toreplace=toreplace.replace('_',' ')
            #print("TOREPLACE: {}".format(toreplace))
            #word is replaced with the candidate accompanying "#" characters as boundaries
            #this is done to prevent the replacement of the in-word and multiple occurences of the words in process 
            pattern='(?<![a-zA-Z#])'+word+'(?![a-zA-Z#])'
            linemodtotal=re.sub(pattern,'#'+toreplace+'#',linemodtotal,1)
            #print(linemodtotal)
            changed+=1
        else:
            continue
        #if the desired number of words are replaced
        if changed==nchange[0]:
            #indicator serves as a sign of the capitalized sentence
            #the "#" characters are removed at the end
            if (indicator):
                toadd=linemodtotal.replace('#','').capitalize()
            else:
                toadd=linemodtotal.replace('#','')
            #the augmented sentences added to the list defined at the beginning
            noisesent6.append(toadd)
            break

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [10]:
#the path where the document containing the augmented samples will be saved to
FINALPATH="/content/"

noiselist=[noisesent6]
pername=["0.6"]
for index in range(0,len(noiselist)):
    augmentedset=noiselist[index]
    #stating the percentage value name that the document containing the augmented maxims will be saved
    folder=pername[index]
    #array that contains the augmented maxims
    final=[]
    start=0
    #merging the sentences that belong to the same maxim again
    for i in range(0,len(nsentence)):
        toaddm=''
        n=nsentence[i]
        for j in range(start,start+n):
            toaddm=toaddm+' '+augmentedset[j]
        toaddm=toaddm[1:]
        final.append(toaddm)
        start=start+n 
    #saving the augmented samples into a text document in the given final path
    f=open(FINALPATH+'/maximsamples_wordnet2pos'+folder[-1]+'0.txt', 'w',encoding="utf8")
    for line in final:
        f.write(line+'\n')
    f.close()